<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/covid19_tweet/rnn%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [446]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import re
import nltk
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('validation.csv')

  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 35.3MB/s]
  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 82.7MB/s]


In [447]:
train_df.head()

,ID,Tweet,Labels
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10
1,2,BanMediaHouse whose is responsible for spreadi...,6
2,3,Im waiting for someone to say to me that all t...,3 4
3,4,He is a liar. Proven day night. Time again. Li...,6
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8


In [448]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [449]:
train_df['label']  = train_df['Labels'].apply(lambda x :  labelsplit(x))
train_df.head(2)

,ID,Tweet,Labels,label
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10,"[0, 10]"
1,2,BanMediaHouse whose is responsible for spreadi...,6,[6]


In [450]:
onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in train_df.label[i] : # j = value   
        onehot[i,j]  = 1

onehot = pd.DataFrame(onehot, columns=['0','1','2','3','4','5','6','7','8','9','10'])
onehot[:3]

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [451]:
train_df = pd.concat(
    [train_df, onehot], axis=1
).drop(['ID','Labels', 'label'],axis=1)

train_df.head(2)

,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [452]:
train = train_df[:3500]
val = train_df[3500:]

In [453]:
train.head(3)

,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Im waiting for someone to say to me that all t...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [454]:
val.head(3)

,Tweet,0,1,2,3,4,5,6,7,8,9,10
3500,Another record daily rise in coronavirus cases...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3501,"PSA: BE THANKFUL FOR WHAT YOU HAVE, SHUT UP, E...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3502,Former USSR and US are United against Chinese ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [455]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [456]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [457]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [458]:
stop_word = stopwords.words('english')
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [459]:
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_tokenize(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_lemmatizer(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x))
    return df

In [460]:
train_clean = clean_text(train, 'Tweet', 'clean_tweet')
val_clean = clean_text(val, 'Tweet', 'clean_tweet')

In [461]:
train_clean

,Tweet,0,1,2,3,4,5,6,7,8,9,10,clean_tweet
0,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,joke hop plane rn well covid lol
1,BanMediaHouse whose is responsible for spreadi...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,banmediahouse whose responsible spreading fake...
2,Im waiting for someone to say to me that all t...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,im waiting someone say corona thing april fool...
3,He is a liar. Proven day night. Time again. Li...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,liar proven day night time lie truth covid
4,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,new u coronavirus death toll reach nearly new ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,Corona Virus please go away I took going out f...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,corona virus please go away took going granted...
3496,19.taxes+phone bill+rent+car payment+no job=no...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,taxesphone billrentcar paymentno jobno stimulu...
3497,"My dog had a dry cough over Christmas, I recko...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,dog dry cough christmas reckon coronavirus swe...
3498,Have you or anyone you know been affected by C...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,anyone know affected covid tested positive rec...


In [462]:
train = train_clean[cols_target]
train['Tweet'] =  train_clean['clean_tweet']
train.head(3)

,0,1,2,3,4,5,6,7,8,9,10,Tweet
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,joke hop plane rn well covid lol
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,banmediahouse whose responsible spreading fake...
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,im waiting someone say corona thing april fool...


In [463]:
val = val_clean[cols_target]
val['Tweet'] = val_clean['clean_tweet']
val.head(3)

,0,1,2,3,4,5,6,7,8,9,10,Tweet
3500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,another record daily rise coronavirus case ann...
3501,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,psa thankful shut enjoy boring as time home covid
3502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,former ussr u united chinese originated viral ...


In [464]:
x_train = train.Tweet.copy()
x_test = val.Tweet.copy()

#y_train = train

In [465]:
y_train = train[cols_target[0]]
y_test = val[cols_target[0]]

In [466]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((3500,), (3500,), (1500,), (1500,))

In [467]:
max_len = np.max(x_train.apply(lambda x : len(x)))
max_len

120

In [468]:
# 토큰화
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
vocab_length = len(tokenizer.word_index) + 1

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

x_train = pad_sequences(x_train, maxlen= max_len, padding='post')
x_test = pad_sequences(x_test, maxlen= max_len, padding='post')

In [469]:
print(vocab_length)
print(max_len)

6896
120


In [470]:
embedding_dim = 16

In [471]:
y_train = to_categorical(y_train,2)
y_train

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [472]:
y_test = to_categorical(y_test,2)
y_test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [478]:
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_length, embedding_dim, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])

In [479]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical


In [480]:
y_train.shape

(3500, 2)

In [481]:
num_epochs = 300
model.fit(x_train, y_train, epochs = num_epochs,
          validation_data = (x_test, y_test),
          verbose = 1)

Epoch 1/300
110/110 [==============================] - 15s 110ms/step - loss: 0.6574 - accuracy: 0.7489 - val_loss: 0.5867 - val_accuracy: 0.7740
Epoch 2/300
110/110 [==============================] - 12s 105ms/step - loss: 0.6050 - accuracy: 0.7597 - val_loss: 0.5559 - val_accuracy: 0.7740
Epoch 3/300
110/110 [==============================] - 12s 106ms/step - loss: 0.5906 - accuracy: 0.7597 - val_loss: 0.5629 - val_accuracy: 0.7740
Epoch 4/300
110/110 [==============================] - 12s 106ms/step - loss: 0.5711 - accuracy: 0.7600 - val_loss: 0.5261 - val_accuracy: 0.7747
Epoch 5/300
110/110 [==============================] - 12s 106ms/step - loss: 0.4600 - accuracy: 0.8063 - val_loss: 0.5133 - val_accuracy: 0.7473
Epoch 6/300
110/110 [==============================] - 12s 107ms/step - loss: 0.4206 - accuracy: 0.8437 - val_loss: 0.5125 - val_accuracy: 0.7620
Epoch 7/300
110/110 [==============================] - 12s 107ms/step - loss: 0.3606 - accuracy: 0.8700 - val_loss: 0.5754 -

KeyboardInterrupt: ignored